1. Посчитайте, сколько компаний закрылось.

In [ ]:
SELECT COUNT(*)
FROM company
WHERE status = 'closed';

2. Отобразите количество привлечённых средств для новостных компаний США. Используйте данные из таблицы company. Отсортируйте таблицу по убыванию значений в поле funding_total .

In [ ]:
SELECT funding_total
FROM company
WHERE category_code = 'news' AND
country_code = 'USA'
ORDER BY funding_total DESC;

3. Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.

In [ ]:
SELECT SUM(price_amount)
FROM acquisition
WHERE term_code = 'cash' AND
EXTRACT(YEAR FROM CAST(acquired_at AS date)) >= 2011 AND 
EXTRACT(YEAR FROM CAST(acquired_at AS date)) <= 2013
;

4. Отобразите имя, фамилию и названия аккаунтов людей в твиттере, у которых названия аккаунтов начинаются на 'Silver'.

In [ ]:
SELECT first_name,
       last_name,
       twitter_username
FROM people
WHERE twitter_username LIKE 'Silver%';

5. Выведите на экран всю информацию о людях, у которых названия аккаунтов в твиттере содержат подстроку 'money', а фамилия начинается на 'K'.

In [ ]:
SELECT *
FROM people
WHERE twitter_username LIKE '%money%' AND
      last_name LIKE 'K%'

;

6. Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.

In [ ]:
SELECT country_code,
       SUM(funding_total)
FROM company
GROUP BY country_code
ORDER BY sum desc
;

7. Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату.
Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.

In [ ]:
SELECT funded_at, MIN(raised_amount),
       MAX(raised_amount)
FROM funding_round
GROUP BY funded_at
HAVING MIN(raised_amount) != MAX(raised_amount)
AND MIN(raised_amount) > 0;

8. Создайте поле с категориями:
Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию high_activity.
Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию middle_activity.
Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию low_activity.
Отобразите все поля таблицы fund и новое поле с категориями.

In [ ]:
SELECT *,
CASE
    WHEN invested_companies >= 100 THEN 'high_activity'
    WHEN invested_companies BETWEEN 20 AND 99 THEN 'middle_activity'
    WHEN invested_companies < 20 THEN 'low_activity'
END
FROM fund;

9. Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.

In [ ]:
SELECT t.activity,
       ROUND(AVG(t.investment_rounds))
FROM
(SELECT *,
       CASE
           WHEN invested_companies>=100 THEN 'high_activity'
           WHEN invested_companies>=20 THEN 'middle_activity'
           ELSE 'low_activity'
       END AS activity
FROM fund) AS t
GROUP BY activity
ORDER BY round;

10. Проанализируйте, в каких странах находятся фонды, которые чаще всего инвестируют в стартапы. 
Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды этой страны, основанные с 2010 по 2012 год включительно. Исключите страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю. Выгрузите десять самых активных стран-инвесторов.
Отсортируйте таблицу по среднему количеству компаний от большего к меньшему, а затем по коду страны в лексикографическом порядке.

In [ ]:
SELECT country_code,
       MIN(invested_companies),
       MAX(invested_companies),
       AVG(invested_companies)
FROM fund
WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) BETWEEN '2010' AND '2012' 
GROUP BY country_code
HAVING MIN(invested_companies) > 0
ORDER BY AVG(invested_companies) DESC, country_code
limit 10;

11. Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.

In [ ]:
SELECT ppl.first_name,
ppl.last_name,
ed.instituition
FROM people AS ppl
LEFT JOIN education AS ed ON ppl.id=ed.person_id

12. Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов.

In [ ]:
WITH
p AS (
      SELECT *
      FROM people),
c AS (
      SELECT *
      FROM company),
ed AS (
      SELECT *
      FROM education)
SELECT c.name,
       COUNT(DISTINCT ed.instituition)
FROM c LEFT JOIN p ON c.id = p.company_id
LEFT JOIN ed ON p.id = ed.person_id
GROUP BY c.name
ORDER BY COUNT(DISTINCT ed.instituition) DESC
LIMIT 5
;

13. Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.

In [ ]:
select distinct name
from company
where id in (select company_id
from funding_round
where is_first_round = 1 and is_first_round = is_last_round) and status = 'closed';

14. Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.

In [ ]:
WITH
i AS (select distinct id
from company
where id in (select company_id
from funding_round
where is_first_round = 1 and is_first_round = is_last_round) and status = 'closed')
SELECT DISTINCT p.id
FROM people AS p
INNER JOIN i ON i.id = p.company_id
;

15. Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.

In [ ]:
WITH
i AS (select distinct id
from company
where id in (select company_id
from funding_round
where is_first_round = 1 and is_first_round = is_last_round) and status = 'closed')

SELECT DISTINCT p.id, e.instituition
FROM people AS p
INNER JOIN i ON i.id = p.company_id
INNER JOIN education AS e ON p.id = e.person_id
;

16. Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания. При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды.

In [ ]:
WITH
i AS (select distinct id
from company
where id in (select company_id
from funding_round
where is_first_round = 1 and is_first_round = is_last_round) and status = 'closed')

SELECT p.id, COUNT(e.instituition)
FROM people AS p
INNER JOIN i ON i.id = p.company_id
INNER JOIN education AS e ON p.id = e.person_id
group by p.id
;

17. Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.

In [ ]:
WITH
i AS (select distinct id
from company
where id in (select company_id
from funding_round
where is_first_round = 1 and is_first_round = is_last_round) and status = 'closed')

SELECT AVG(d.count)
FROM
(SELECT p.id, COUNT(e.instituition)
FROM people AS p
INNER JOIN i ON i.id = p.company_id
INNER JOIN education AS e ON p.id = e.person_id
group by p.id) AS d
;

18. Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Facebook*.
*(сервис, запрещённый на территории РФ)

In [ ]:
WITH companies AS
    (SELECT DISTINCT id
    FROM company
    WHERE name='Facebook')

SELECT AVG(ff.count)
FROM (
SELECT p.id as id,
(COUNT( ed.instituition)) as count
from people as p
JOIN education as ed  ON p.id=ed.person_id
Where company_id in (select id from companies)
GROUP BY p.id) as ff

19. Составьте таблицу из полей:
name_of_fund — название фонда;
name_of_company — название компании;
amount — сумма инвестиций, которую привлекла компания в раунде.
В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.

In [ ]:
SELECT fund.name as name_of_fund, company.name as name_of_company, funding_round.raised_amount as amount
FROM investment
INNER JOIN fund ON fund.id=investment.fund_id
INNER JOIN funding_round ON funding_round.id=investment.funding_round_id
INNER JOIN company ON company.id=investment.company_id
Where company.id in (select id from company where milestones > 6)
And funding_round.id in (select id from funding_round where extract(YEAR FROM CAST(funded_at as DATE)) between 2012 and 2013)

20. Выгрузите таблицу, в которой будут такие поля:
название компании-покупателя;
сумма сделки;
название компании, которую купили;
сумма инвестиций, вложенных в купленную компанию;
доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.
Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы. 
Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями.

In [ ]:
WITH table_ AS(
               SELECT t1.*, t2.*
               FROM acquisition AS t1             
               LEFT JOIN company AS t2 ON t1.acquiring_company_id = t2.id)  
SELECT t1.name,
       t1.price_amount,
       t2.name,
       t2.funding_total,
       ROUND(t1.price_amount / t2.funding_total)
FROM table_ AS t1
LEFT JOIN company t2 ON t1.acquired_company_id = t2.id
WHERE t1.price_amount > 0 AND t2.funding_total > 0
ORDER BY t1.price_amount DESC, t2.name
LIMIT 10;

21. Выгрузите таблицу, в которую войдут названия компаний из категории social, получившие финансирование с 2010 по 2013 год включительно. Проверьте, что сумма инвестиций не равна нулю. Выведите также номер месяца, в котором проходил раунд финансирования.

In [ ]:
WITH
t1 AS(SELECT id, name
    FROM company
    WHERE category_code = 'social'),
t2 AS(SELECT company_id, EXTRACT(MONTH FROM CAST(funded_at AS date))
    FROM funding_round
    WHERE raised_amount != 0 AND
    EXTRACT(YEAR FROM CAST(funded_at AS date)) BETWEEN 2010 AND 2013)
    
SELECT t1.name,
       t2.date_part
FROM t2 INNER JOIN t1 ON t1.id = t2.company_id;

22. Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:
номер месяца, в котором проходили раунды;
количество уникальных названий фондов из США, которые инвестировали в этом месяце;
количество компаний, купленных за этот месяц;
общая сумма сделок по покупкам в этом месяце.

In [ ]:
WITH
s1 AS
    (SELECT EXTRACT(MONTH FROM (fr.funded_at)::date) AS month_f,
            COUNT(DISTINCT f.name) AS count_name
    FROM fund as f
    INNER JOIN investment AS i ON f.id = i.fund_id
    INNER JOIN funding_round AS fr ON fr.id = i.funding_round_id
    WHERE f.country_code = 'USA' AND
          EXTRACT(YEAR FROM (funded_at)::date) BETWEEN '2010' AND '2013' 
    GROUP BY month_f),
s2 AS
    (SELECT EXTRACT(MONTH FROM (acquired_at)::date) AS month_as,
            count(acquired_company_id) AS count_ac,
            sum(price_amount) AS sum_pr
    FROM acquisition
    WHERE EXTRACT(YEAR FROM (acquired_at)::date) BETWEEN '2010' AND '2013' 
    GROUP BY month_as)
    
SELECT s2.month_as,
       count_name,
       count_ac,
       sum_pr
FROM s1
JOIN s2 ON s1.month_f = s2.month_as

23. Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.

In [ ]:
WITH
s AS (SELECT g.country_code
      FROM
          (SELECT DISTINCT country_code, EXTRACT(YEAR FROM CAST(founded_at AS date))
           FROM company
           WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) BETWEEN 2011 AND 2013 AND
               country_code IS NOT NULL
           ORDER BY country_code) AS g
      GROUP BY g.country_code
      HAVING AVG(g.date_part) = 2012),
      
t2011 AS(
SELECT c.country_code,
       AVG(f.raised_amount)
FROM funding_round AS f
INNER JOIN company AS c ON c.id = f.company_id
WHERE EXTRACT(YEAR FROM CAST(c.founded_at AS date)) = 2011 AND
    c.country_code IS NOT NULL AND
    funding_total >0
GROUP BY c.country_code,
         EXTRACT(YEAR FROM CAST(c.founded_at AS date))),

t2012 AS(
SELECT c.country_code,
       AVG(f.raised_amount)
FROM funding_round AS f
INNER JOIN company AS c ON c.id = f.company_id
WHERE EXTRACT(YEAR FROM CAST(c.founded_at AS date)) = 2012 AND
    c.country_code IS NOT NULL AND
    funding_total >0
GROUP BY c.country_code,
         EXTRACT(YEAR FROM CAST(c.founded_at AS date))),

t2013 AS(
SELECT c.country_code,
       AVG(f.raised_amount)
FROM funding_round AS f
INNER JOIN company AS c ON c.id = f.company_id
WHERE EXTRACT(YEAR FROM CAST(c.founded_at AS date)) = 2013 AND
    c.country_code IS NOT NULL AND
    funding_total >0
GROUP BY c.country_code,
         EXTRACT(YEAR FROM CAST(c.founded_at AS date)))

SELECT t1.country_code,
       t1.avg,
       t2.avg,
       t3.avg
FROM t2011 AS t1
INNER JOIN t2012 AS t2 ON t1.country_code = t2.country_code
INNER JOIN t2013 AS t3 ON t1.country_code = t3.country_code
INNER JOIN s ON s.country_code = t1.country_code
ORDER BY t1.avg DESC;